# Исследование данных об инвестиции венчурных фондов в компании-стартапы

### Описание проекта

В данном самостоятельном проекте нужно проанализировать данные о фондах и инвестициях и написать запросы к базе.

По данной [ссылке](https://drive.google.com/file/d/1JT7J_ZNmGBiD6jf1KJ2GT1d1CHqfFy18/view?usp=sharing) можно ознакомиться с ER-диаграммой и описанием таблиц в базе данных.

Ниже будут перечислены задачи с их описанием, а ниже описания задачи будет приведено решение задачи в виде запроса на языке SQL.

<hr>

**1. Посчитайте, сколько компаний закрылось.**

**SQL-запрос решения:**

`SELECT COUNT(id) AS closed_count
FROM company
WHERE closed_at IS NOT NULL
    AND status = 'closed'`

<hr>

**2. Отобразите количество привлечённых средств для новостных компаний США. Используйте данные из таблицы `company`. Отсортируйте таблицу по убыванию значений в поле `funding_total`.**

**SQL-запрос решения:**

`SELECT SUM(funding_total) AS funding_total
FROM company
WHERE category_code = 'news'
    AND country_code = 'USA'
GROUP BY name
ORDER BY funding_total DESC`

<hr>

**3. Найдите общую сумму сделок по покупке одних компаний другими в долларах. Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.**

**SQL-запрос решения:**

`SELECT SUM(price_amount)
FROM acquisition
WHERE term_code = 'cash'
    AND EXTRACT(YEAR FROM CAST(acquired_at AS date)) BETWEEN 2011 AND 2013`

<hr>

**4. Отобразите имя, фамилию и названия аккаунтов людей в твиттере, у которых названия аккаунтов начинаются на `'Silver'`.**

**SQL-запрос решения:**

`SELECT  first_name,
        last_name,
        twitter_username
FROM people
WHERE twitter_username LIKE 'Silver%'`

<hr>

**5. Выведите на экран всю информацию о людях, у которых названия аккаунтов в твиттере содержат подстроку `'money'`, а фамилия начинается на `'K'`.**

**SQL-запрос решения:**

`SELECT *
FROM people
WHERE last_name LIKE 'K%'
    AND twitter_username LIKE '%money%'`

<hr>

**6. Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы.**

**SQL-запрос решения:**

`SELECT country_code,
    SUM(funding_total) as total
FROM company
GROUP BY country_code
ORDER BY total DESC`

<hr>

**7. Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату.**

Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.

**SQL-запрос решения:**

`SELECT funded_at,
    MIN(raised_amount),
    MAX(raised_amount)
FROM funding_round
GROUP BY funded_at
HAVING MIN(raised_amount) != 0
    AND MIN(raised_amount) != MAX(raised_amount)`

<hr>

**8. Создайте поле с категориями:**
- Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию `high_activity`.
- Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию `middle_activity`.
- Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию `low_activity`.

Отобразите все поля таблицы `fund` и новое поле с категориями.

**SQL-запрос решения:**

`SELECT *,
    CASE
        WHEN invested_companies >= 100 THEN 'high_activity'
        WHEN invested_companies >= 20 THEN 'middle_activity'
        ELSE 'low_activity'   
    END
FROM fund`

<hr>

**9. Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие. Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.**

**SQL-запрос решения:**

`WITH
a AS (SELECT *,
       CASE
           WHEN invested_companies>=100 THEN 'high_activity'
           WHEN invested_companies>=20 THEN 'middle_activity'
           ELSE 'low_activity'
       END AS activity
FROM fund)
SELECT a.activity,
    ROUND(AVG(a.investment_rounds), 0) as avg_rounds
FROM a
GROUP BY a.activity
ORDER BY avg_rounds`

<hr>

**10. Проанализируйте, в каких странах находятся фонды, которые чаще всего инвестируют в стартапы.**  

Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды этой страны, основанные с 2010 по 2012 год включительно. Исключите страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю. 

Выгрузите десять самых активных стран-инвесторов: отсортируйте таблицу по среднему количеству компаний от большего к меньшему. Затем добавьте сортировку по коду страны в лексикографическом порядке.

**SQL-запрос решения:**

`SELECT country_code,
              MIN(invested_companies) AS min_inv,
              MAX(invested_companies) AS max_inv,
              AVG(invested_companies) AS avg_inv
        FROM fund
        WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) BETWEEN 2010 AND 2012
        GROUP BY country_code
        HAVING MIN(invested_companies) != 0
        ORDER BY avg_inv DESC
        LIMIT 10`

<hr>

**11. Отобразите имя и фамилию всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.**

**SQL-запрос решения:**

`SELECT p.first_name,
    p.last_name,
    e.instituition
FROM people AS p
LEFT JOIN education AS e ON e.person_id = p.id`

<hr>

**12. Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники. Выведите название компании и число уникальных названий учебных заведений. Составьте топ-5 компаний по количеству университетов.**

**SQL-запрос решения:**

`WITH
d AS (SELECT *
    FROM company AS c
    LEFT JOIN people AS p ON p.company_id = c.id
    LEFT JOIN education AS e ON e.person_id = p.id
)
SELECT d.name,
    COUNT(DISTINCT d.instituition) AS count_inst
FROM d
GROUP BY d.name
ORDER BY count_inst DESC
LIMIT 5;`

<hr>

**13. Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.**

**SQL-запрос решения:**

`SELECT DISTINCT c.name
FROM company AS c
LEFT JOIN funding_round AS fr ON fr.company_id = c.id
WHERE c.status = 'closed'
    AND c.closed_at IS NOT NULL
    AND fr.is_first_round = 1
    AND fr.is_first_round = fr.is_last_round`

<hr>

**14. Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.**

**SQL-запрос решения:**

`WITH
cc AS (SELECT DISTINCT c.id AS id
        FROM company AS c
        LEFT JOIN funding_round AS fr ON fr.company_id = c.id
        WHERE c.status = 'closed'
            AND c.closed_at IS NOT NULL
            AND fr.is_first_round = 1
            AND fr.is_first_round = fr.is_last_round)
SELECT p.id
FROM people AS p
LEFT JOIN cc AS c ON c.id = p.company_id
WHERE c.id IS NOT NULL`

<hr>

**15. Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.**

**SQL-запрос решения:**

`WITH
cc AS (SELECT DISTINCT c.id AS id
        FROM company AS c
        LEFT JOIN funding_round AS fr ON fr.company_id = c.id
        WHERE c.status = 'closed'
            AND c.closed_at IS NOT NULL
            AND fr.is_first_round = 1
            AND fr.is_first_round = fr.is_last_round)
SELECT p.id,
        e.instituition
FROM people AS p
LEFT JOIN cc AS c ON c.id = p.company_id
LEFT JOIN education AS e ON e.person_id = p.id
WHERE c.id IS NOT NULL
GROUP BY p.id, e.instituition
HAVING e.instituition IS NOT NULL`

<hr>

**16. Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания. При подсчёте учитывайте, что некоторые сотрудники могли окончить одно и то же заведение дважды.**

**SQL-запрос решения:**

`WITH
cc AS (SELECT DISTINCT c.id AS id
        FROM company AS c
        LEFT JOIN funding_round AS fr ON fr.company_id = c.id
        WHERE c.status = 'closed'
            AND c.closed_at IS NOT NULL
            AND fr.is_first_round = 1
            AND fr.is_first_round = fr.is_last_round)
SELECT p.id,
        COUNT(e.instituition)
FROM people AS p
LEFT JOIN cc AS c ON c.id = p.company_id
LEFT JOIN education AS e ON e.person_id = p.id
WHERE c.id IS NOT NULL
GROUP BY p.id
HAVING COUNT(e.instituition) != 0`

<hr>

**17. Дополните предыдущий запрос и выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. Нужно вывести только одну запись, группировка здесь не понадобится.**

**SQL-запрос решения:**

`WITH
cc AS (SELECT DISTINCT c.id AS id
        FROM company AS c
        LEFT JOIN funding_round AS fr ON fr.company_id = c.id
        WHERE c.status = 'closed'
            AND c.closed_at IS NOT NULL
            AND fr.is_first_round = 1
            AND fr.is_first_round = fr.is_last_round),
cp AS(SELECT p.id,
        COUNT(e.instituition)
FROM people AS p
LEFT JOIN cc AS c ON c.id = p.company_id
LEFT JOIN education AS e ON e.person_id = p.id
WHERE c.id IS NOT NULL
GROUP BY p.id
HAVING COUNT(e.instituition) != 0)
SELECT AVG(count)
FROM cp`

<hr>

**18. Составьте таблицу из полей:**
- `name_of_fund` — название фонда;
- `name_of_company` — название компании;
- `amount` — сумма инвестиций, которую привлекла компания в раунде.

В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.

**SQL-запрос решения:**

`SELECT f.name AS name_of_fund,
        c.name AS name_of_company,
        fr.raised_amount AS amount
FROM investment AS i
LEFT JOIN company AS c ON c.id = i.company_id
LEFT JOIN fund AS f ON f.id = i.fund_id
JOIN funding_round AS fr ON fr.id = i.funding_round_id
WHERE c.milestones > 6
    AND EXTRACT(YEAR FROM CAST(fr.funded_at AS date)) BETWEEN 2012 AND 2013
    AND fr.raised_amount > 0`

<hr>

**19. Выгрузите таблицу, в которой будут такие поля:**
- название компании-покупателя;
- сумма сделки;
- название компании, которую купили;
- сумма инвестиций, вложенных в купленную компанию;
- доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.

Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы. 

Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Ограничьте таблицу первыми десятью записями.

**SQL-запрос решения:**

`SELECT c1.name AS acquiring,
    a.price_amount AS price,
    c2.name AS acquired,
    c2.funding_total AS investment,
    ROUND(a.price_amount/c2.funding_total, 0) AS roi
FROM acquisition AS a
LEFT JOIN company as c1 on c1.id = a.acquiring_company_id
LEFT JOIN company as c2 on c2.id = a.acquired_company_id
WHERE a.price_amount > 0
    AND c2.funding_total > 0
ORDER BY price DESC, acquired
LIMIT 10`

<hr>

**20. Выгрузите таблицу, в которую войдут названия компаний из категории `social`, получившие финансирование с 2010 по 2013 год включительно. Проверьте, что сумма инвестиций не равна нулю. Выведите также номер месяца, в котором проходил раунд финансирования.**

**SQL-запрос решения:**

`SELECT c.name,
      EXTRACT(MONTH FROM CAST(fr.funded_at AS date)) AS month
FROM company AS c
LEFT JOIN funding_round AS fr ON fr.company_id = c.id
WHERE c.category_code = 'social'
    AND EXTRACT(YEAR FROM CAST(fr.funded_at AS date)) BETWEEN 2010 AND 2013
`

<hr>

**21. Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:**
- номер месяца, в котором проходили раунды;
- количество уникальных названий фондов из США, которые инвестировали в этом месяце;
- количество компаний, купленных за этот месяц;
- общая сумма сделок по покупкам в этом месяце.

**SQL-запрос решения:**

`WITH
fm AS (SELECT DISTINCT EXTRACT(MONTH FROM CAST(funded_at AS date)) AS month
      FROM funding_round
      WHERE EXTRACT(YEAR FROM CAST(funded_at AS date)) BETWEEN 2010 AND 2013),
fr_id AS (SELECT id,
          EXTRACT(MONTH FROM CAST(funded_at AS date)) AS month
         FROM funding_round
         WHERE EXTRACT(YEAR FROM CAST(funded_at AS date)) BETWEEN 2010 AND 2013),
f_usa AS (SELECT fr.month AS month,
                  COUNT(DISTINCT f.name) AS count
         FROM fund AS f
         JOIN investment AS i ON i.fund_id = f.id
         JOIN fr_id AS fr ON fr.id = i.funding_round_id
         WHERE f.country_code = 'USA'
         GROUP BY fr.month),
ac AS (SELECT EXTRACT(MONTH FROM CAST(acquired_at AS date)) AS month,
       COUNT(acquired_company_id) AS count,
       SUM(price_amount) AS sum
      FROM acquisition
      WHERE EXTRACT(YEAR FROM CAST(acquired_at AS date)) BETWEEN 2010 AND 2013
      GROUP BY month)
SELECT fm.month,
        f.count,
        ac.count,
        ac.sum
FROM fm AS fm
LEFT JOIN f_usa AS f ON f.month = fm.month
LEFT JOIN ac AS ac ON ac.month = fm.month`

<hr>

**22. Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.**

**SQL-запрос решения:**

`WITH
cc AS (SELECT DISTINCT country_code
      FROM fund),
i_2011 AS (SELECT country_code,
          AVG(funding_total) AS avg
          FROM company
          WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = 2011
          GROUP BY country_code),
i_2012 AS (SELECT country_code,
          AVG(funding_total) AS avg
          FROM company
          WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = 2012
          GROUP BY country_code),
i_2013 AS (SELECT country_code,
          AVG(funding_total) AS avg
          FROM company
          WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = 2013
          GROUP BY country_code),
a_2011 AS (SELECT f.country_code AS country_code,
               AVG(fr.raised_amount) as avg
          FROM fund AS f
          JOIN investment AS i ON i.fund_id = f.id
          JOIN funding_round AS fr ON fr.id = i.funding_round_id
          WHERE f.country_code IN (SELECT country_code FROM i_2011)
              AND EXTRACT(YEAR FROM CAST(fr.funded_at AS date)) = 2011
               AND fr.raised_amount > 0
          GROUP BY  f.country_code),
a_2012 AS (SELECT f.country_code AS country_code,
               AVG(fr.raised_amount) as avg
          FROM fund AS f
          JOIN investment AS i ON i.fund_id = f.id
          JOIN funding_round AS fr ON fr.id = i.funding_round_id
          WHERE f.country_code IN (SELECT country_code FROM i_2012)
              AND EXTRACT(YEAR FROM CAST(fr.funded_at AS date)) = 2012
           AND fr.raised_amount > 0
          GROUP BY  f.country_code),
a_2013 AS (SELECT f.country_code AS country_code,
           AVG(fr.raised_amount) as avg
          FROM fund AS f
          JOIN investment AS i ON i.fund_id = f.id
          JOIN funding_round AS fr ON fr.id = i.funding_round_id
          WHERE f.country_code IN (SELECT country_code FROM i_2013)
              AND EXTRACT(YEAR FROM CAST(fr.funded_at AS date)) = 2013
           AND fr.raised_amount > 0
          GROUP BY f.country_code)
SELECT i_2011.country_code AS country,
        i_2011.avg AS avg_2011,
        i_2012.avg AS avg_2012,
        i_2013.avg AS avg_2013
FROM i_2011
JOIN i_2012 ON i_2012.country_code = i_2011.country_code
JOIN i_2013 ON i_2013.country_code = i_2012.country_code
ORDER BY avg_2011 DESC`

<hr>